# Exploración de variables espaciales
<b>Proyecto</b>:	PDB IV - Pobreza urbana
<b>Objetivo</b>:    Desarrollo de variables espaciales y tabulaciones de pobreza con variables 

### 1. Bases de datos y librerías

In [3]:
import pandas as pd
import os
import numpy as np
import geopandas as gdp
import matplotlib.pyplot as plt
from geopy.distance import geodesic

In [5]:
os.chdir(r'C:\Users\User\OneDrive - MIGRACIÓN VIDENZA\1. Proyectos\1. Proyectos actuales\23. Artículos PDB\2. PDB - Pobreza Urbana\2. Data\1. Bases\2. ENAHO Anual')

mod5 = pd.read_stata('modulo500.dta', convert_categoricals=False,
                     columns= ['anio', 'mes', 'conglome', 'vivienda','hogar', 'ubigeo', 'dominio', 'estrato',
                               'codperso', 'fac500a', 'p558d2_1', 'p558d2_2', 'ocupinf', 'i524a1', 'd529t',
                               'i530a', 'd536', 'i538a1', 'd540t', 'i541a', 'd543', 'd544t', 'p521a', 'p521',
                               'p519','i520', 'i513t', 'i518'])

sumaria = pd.read_stata('sumaria.dta', convert_categoricals=False,
                        columns= ['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'percepho', 'mieperho', 'gashog2d',
                                  'linpe', 'linea', 'gashog1d', 'pobrezav', 'pobreza', 'ipcr_0', 'gpgru0'])

mod1 = pd.read_stata('modulo100.dta', convert_categoricals=False,
                     columns= ['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'latitud', 'longitud'])

mod2 = pd.read_stata('modulo200.dta', convert_categoricals=False,
                     columns=['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'codperso', 'p208a'])

dfEnaho = pd.merge(mod5, sumaria, how='inner', on=['anio', 'mes', 'conglome', 'vivienda', 'hogar'], validate="m:1")

dfEnaho = pd.merge(dfEnaho, mod1, how='inner', on=['anio', 'mes', 'conglome', 'vivienda', 'hogar'], validate="m:1")


C:\Users\User\AppData\Local\Temp\ipykernel_20436\3645476878.py:3: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  mod5 = pd.read_stata('modulo500.dta', convert_categoricals=False,
C:\Users\User\AppData\Local\Temp\ipykernel_20436\3645476878.py:9: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  sumaria = pd.read_stata('sumaria.dta', convert_categoricals=False,
C:\Users\User\AppData\Local\Temp\ipykernel_20436\3645476878.py:13: UnicodeWarning: 
One or more strings in the dta file could not be decoded using ut

In [38]:
# Nos quedamos solo con las observaciones urbanas
dfUrb = dfEnaho[dfEnaho['estrato'] < 6]

## 2. Exploración distancia al trabajo y al centro de la ciudad

In [39]:
# Modificamos la variable p558d2_2 para ponerlo en formato ubigeo
dfUrb.loc[:,'p558d2_2'] = dfUrb['p558d2_2'].fillna('0')
dfUrb.loc[:,'p558d2_2'] = dfUrb['p558d2_2'].astype(int).astype(str).str.zfill(6)

C:\Users\User\AppData\Local\Temp\ipykernel_26316\830759936.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0' '0' '0' ... 150811.0 150811.0 '0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  dfUrb.loc[:,'p558d2_2'] = dfUrb['p558d2_2'].fillna('0')


#### 2.1. Residencia en distritos atractores de empleo

In [40]:
# Generamos la variable de pctResidencia
distResidencia = dfUrb.loc[:,['ubigeo','fac500a']].dropna()
distResidencia['provResidencia'] = (distResidencia['ubigeo'].astype(int) / 100).astype(int).astype(str).str.zfill(4)
distResidencia = distResidencia.dropna()

# Contabilizamos la cantidad de personas que residen por distrito
distResidencia = distResidencia.groupby(['provResidencia',"ubigeo"])['fac500a'].sum().reset_index()

distResidencia = distResidencia.rename(columns={'fac500a':'nPersonas'})
distResidencia['nPersonasProv'] = distResidencia.groupby('provResidencia')['nPersonas'].transform('sum')
distResidencia['pctResidencia'] = distResidencia['nPersonas'] / distResidencia['nPersonasProv']

In [41]:
# Generamos la variable ubigeo del centro de trabajo
dfUrb.loc[:,'ubigeoTrabajo'] = dfUrb.apply(lambda row: row['ubigeo'] if row['p558d2_1'] == 1 else row['p558d2_2'], axis=1)
dfUrb.loc[:,'ubigeoTrabajo'] = dfUrb['ubigeoTrabajo'].replace("000000", pd.NA).replace("999999", pd.NA)

C:\Users\User\AppData\Local\Temp\ipykernel_26316\3557774175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfUrb.loc[:,'ubigeoTrabajo'] = dfUrb.apply(lambda row: row['ubigeo'] if row['p558d2_1'] == 1 else row['p558d2_2'], axis=1)


In [42]:
# Generamos la variable provincia del trabajo 
distTrabajo = dfUrb.loc[:,['ubigeoTrabajo','ubigeo','fac500a']].dropna()
distTrabajo['provTrabajo'] = (distTrabajo['ubigeoTrabajo'].astype(int) / 100).astype(int).astype(str).str.zfill(4)
distTrabajo = distTrabajo.dropna()

# Contabilizamos la cantidad de empleos que se generan
distTrabajo = distTrabajo.groupby(['provTrabajo',"ubigeoTrabajo"])['fac500a'].sum().reset_index()

In [43]:
distTrabajo = distTrabajo.rename(columns={'fac500a':'nTrabajos'})
distTrabajo['nTrabajosProv'] = distTrabajo.groupby('provTrabajo')['nTrabajos'].transform('sum')
distTrabajo['pctTrabajo'] = distTrabajo['nTrabajos'] / distTrabajo['nTrabajosProv']

In [44]:
# Definimos como centralidad si un distrito concentra atrae más trabajo que residentes.
distTrabajo = distTrabajo.rename(columns={'ubigeoTrabajo':'ubigeo'})
distUrb = pd.merge(distTrabajo, distResidencia, how='inner', on='ubigeo')
distUrb = distUrb.loc[:, ['ubigeo', 'pctResidencia', 'pctTrabajo']]

cond1 = (distUrb['pctTrabajo'] > distUrb['pctResidencia'] + 0.01 )
distUrb['distCentral'] = 0
distUrb.loc[ cond1 ,'distCentral'] = 1


#### 2.2. Distancia ecluidiana al trabajo

In [45]:
dfUrb = pd.merge(dfUrb, distUrb, how='left', on='ubigeo', validate="m:1")

cond2 = (dfUrb['pobreza'] < 3 )
dfUrb['pobre'] = 0
dfUrb.loc[ cond2 ,'pobre'] = 1
del cond2

In [46]:
tabla1 = pd.crosstab([dfUrb["anio"], dfUrb["distCentral"]], dfUrb["pobre"], values=dfUrb["fac500a"], aggfunc="sum", normalize='index')
tabla1.reset_index(inplace=True)
tabla1 = tabla1.drop(0, axis=1)

In [47]:
path_dist = "C:/Users/User/OneDrive - Universidad del Pacífico/1. Documentos/0. Bases de datos/11. Mapas Perú/Límite Distrital"

peru_dist = gdp.read_file(path_dist + "/LIMITE_DISTRITAL_2020_INEI_geogpsperu_juansuyo_931381206.shp")

var1 = peru_dist['UBIGEO']
var2 = peru_dist.centroid
var3 = var2.x
var4 = var2.y

centroids = pd.DataFrame({'p558d2_2': var1,
                          'x_long' : var3,
                          'y_lat' : var4})
del var1, var2, var3, var4

C:\Users\User\AppData\Local\Temp\ipykernel_26316\2172197495.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  var2 = peru_dist.centroid


In [48]:
dfUrb = pd.merge(dfUrb, centroids, how='left', on='p558d2_2', validate="m:1")

In [49]:
dfUrb2 = dfUrb[dfUrb['p558d2_2'] != '000000']
dfUrb2 = dfUrb2[dfUrb2['y_lat'].notna()]
dfUrb2 = dfUrb2.rename(columns={'y_lat':'lat_trabajo', 'x_long':'long_trabajo'})

In [50]:
dfUrb2 = dfUrb2.dropna(subset=['latitud', 'longitud', 'lat_trabajo', 'long_trabajo'])

dfUrb2.loc[:, 'distanciaTrabajo'] = dfUrb2.apply(lambda row: geodesic((row['latitud'], row['longitud']), (row['lat_trabajo'], row['long_trabajo'])).kilometers, axis=1)

#### 2.3. Distancia ecluidiana al centro de la ciudad

In [51]:
idx = distTrabajo.groupby(['provTrabajo'])['pctTrabajo'].idxmax()
distTrabajo2 = distTrabajo.loc[idx]
distTrabajo2 = distTrabajo2[['provTrabajo', 'ubigeo']]
distTrabajo2 = distTrabajo2.iloc[4:-1,:]

In [52]:
dfUrb2.loc[:,'prov'] = (dfUrb2['ubigeo'].astype(int) / 100).astype(int).astype(str).str.zfill(4)
distTrabajo2 = distTrabajo2.rename(columns={'provTrabajo':'prov'})

dfUrb2 = pd.merge(dfUrb2, distTrabajo2, how='left', on='prov', validate="m:1" )

In [53]:
dfUrb2 = dfUrb2.rename(columns={'ubigeo_y':'ubigeoCentro', 'ubigeo_x':'ubigeo'})

In [54]:
centroids = centroids.rename(columns={'p558d2_2':'ubigeoCentro',
                                      'y_lat':'lat_centro', 'x_long':'long_centro'})
dfUrb2 = pd.merge(dfUrb2, centroids, how='left', on='ubigeoCentro', validate="m:1")

In [55]:

dfUrb2 = dfUrb2[dfUrb2['anio'] >= 2018]

dfUrb2 = dfUrb2.dropna(subset=['latitud', 'longitud', 'lat_centro', 'long_centro'])

dfUrb2.loc[:, 'distanciaCentro'] = dfUrb2.apply(lambda row: geodesic((row['latitud'], row['longitud']), (row['lat_centro'], row['long_centro'])).kilometers, axis=1)


In [56]:
# Definir los límites de los intervalos
bins = [0, 5, 10, 20, 500]

# Definir los nombres de las categorías
labels = ['Menos de 5Km', 'Entre 5 y 10Km', 'Entre 10 y 20Km', 'Más de 20Km']

# Crear la nueva columna categórica
dfUrb2['catDistCentro'] = pd.cut(dfUrb2['distanciaCentro'], bins=bins, labels=labels, right=False)
dfUrb2['catDistTrabajo'] = pd.cut(dfUrb2['distanciaTrabajo'], bins=bins, labels=labels, right=False)


In [57]:
def weighted_mean(df, values, weights):
    return (df[values] * df[weights]).sum() / df[weights].sum()

pobLimaCentro = dfUrb2[dfUrb2['estrato'] == 1].groupby(['anio', 'catDistCentro']).apply(lambda x: weighted_mean(x, 'pobre', 'fac500a'))
pobEst2Centro = dfUrb2[dfUrb2['estrato'] == 2].groupby(['anio', 'catDistCentro']).apply(lambda x: weighted_mean(x, 'pobre', 'fac500a'))


C:\Users\User\AppData\Local\Temp\ipykernel_26316\831550442.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pobLimaCentro = dfUrb2[dfUrb2['estrato'] == 1].groupby(['anio', 'catDistCentro']).apply(lambda x: weighted_mean(x, 'pobre', 'fac500a'))
C:\Users\User\AppData\Local\Temp\ipykernel_26316\831550442.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pobLimaCentro = dfUrb2[dfUrb2['estrato'] == 1].groupby(['anio', 'catDistCentro']).apply(lambda x: weighted_mean(x, 'pobre', 'fac500a'))
C:\Users\User\AppDa

## 3. Exploración distancia a escuelas públicas

In [58]:
os.chdir(r'C:\Users\User\OneDrive - MIGRACIÓN VIDENZA\1. Proyectos\1. Proyectos actuales\23. Artículos PDB\2. PDB - Pobreza Urbana\2. Data\1. Bases')

dfIIEE = pd.read_csv('listado_iiee.csv')
dfIIEE.loc[:,'Ubigeo'] = dfIIEE['Ubigeo'].astype(str).str.zfill(6)
dfIIEE.loc[:,'tipoGestion'] = dfIIEE['Gestion / Dependencia'].str[:7]


C:\Users\User\AppData\Local\Temp\ipykernel_26316\815276184.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['010101' '010101' '010101' ... '250401' '250401' '250401']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dfIIEE.loc[:,'Ubigeo'] = dfIIEE['Ubigeo'].astype(str).str.zfill(6)


In [6]:
os.chdir(r'C:\Users\User\OneDrive - MIGRACIÓN VIDENZA\1. Proyectos\1. Proyectos actuales\23. Artículos PDB\2. PDB - Pobreza Urbana\2. Data\1. Bases\2. ENAHO Anual')
mod1 = pd.read_stata('modulo100.dta', convert_categoricals=False,
                     columns= ['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'latitud', 'longitud', 'estrato', 'ubigeo','factor07'])
mod1 = pd.merge(mod1, sumaria, how='inner', on=['anio', 'mes', 'conglome', 'vivienda', 'hogar'], validate="1:1")

C:\Users\User\AppData\Local\Temp\ipykernel_20436\74732141.py:2: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  mod1 = pd.read_stata('modulo100.dta', convert_categoricals=False,


In [7]:
cond1 = mod1['estrato'] < 6
cond2 = mod1['anio'] >= 2018
mod1 = mod1[cond1 & cond2]

In [61]:
distUrb2 = pd.Series(mod1['ubigeo'].unique(), name='Ubigeo')
dfIEUrb = pd.merge(distUrb2, dfIIEE, on='Ubigeo')
dfIEUrb = dfIEUrb[dfIEUrb['tipoGestion'] == 'Pública']
del distUrb2

In [62]:
dfIEUrb = dfIEUrb.rename(columns={'Ubigeo':'ubigeo'})
dfIEUrb = pd.merge(mod1, dfIEUrb, on='ubigeo', how='inner' , validate='m:m')

In [63]:
dfIEUrb = dfIEUrb.rename(columns={'latitud':'latCong',
                        'longitud' : 'longCong',
                        'Latitud': 'latIE',
                        'Longitud': 'longIE'})

dfIEUrb = dfIEUrb.dropna(subset=['latCong', 'longCong', 'latIE', 'longIE'])

dfIEUrb.loc[:, 'distanciaIE'] = dfIEUrb.apply(lambda row: geodesic((row['latCong'], row['longCong']), (row['latIE'], row['longIE'])).kilometers, axis=1)

In [64]:
dfIEUrb['ID'] = dfIEUrb['anio'].astype(str) + '_' + dfIEUrb['mes'].astype(str) + '_' + dfIEUrb['conglome'].astype(str) + '_' + dfIEUrb['vivienda'].astype(str) + '_' + dfIEUrb['hogar'].astype(str) 

In [65]:
dfIEUrb2 = dfIEUrb.groupby(['ID', 'anio', 'mes', 'conglome', 'vivienda', 'hogar', 'pobreza', 'factor07']).min('distanciaIE')
dfIEUrb2 = dfIEUrb2.reset_index()

In [66]:
# Definir los límites de los intervalos
bins = [0, 0.5, 1, 2.5, 500]

# Definir los nombres de las categorías
labels = ['Menos de 500m', 'Entre 0.5 y 1Km', 'Entre 1 y 2.5 Km', 'Más de 2.5Km']

# Crear la nueva columna categórica
dfIEUrb2['catDistIE'] = pd.cut(dfIEUrb2['distanciaIE'], bins=bins, labels=labels, right=False)

In [8]:
def weighted_mean(df, values, weights):
    return (df[values] * df[weights]).sum() / df[weights].sum()

In [68]:
dfIEUrb2['pobre'] = dfIEUrb2['pobreza']<3

pobLimaIE = dfIEUrb2[dfIEUrb2['estrato'] == 1].groupby(['anio', 'catDistIE']).apply(lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobRestoIE = dfIEUrb2[dfIEUrb2['estrato'] == 2].groupby(['anio', 'catDistIE']).apply(lambda x: weighted_mean(x, 'pobre', 'factor07'))

C:\Users\User\AppData\Local\Temp\ipykernel_26316\520418758.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pobLimaIE = dfIEUrb2[dfIEUrb2['estrato'] == 1].groupby(['anio', 'catDistIE']).apply(lambda x: weighted_mean(x, 'pobre', 'factor07'))
C:\Users\User\AppData\Local\Temp\ipykernel_26316\520418758.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pobLimaIE = dfIEUrb2[dfIEUrb2['estrato'] == 1].groupby(['anio', 'catDistIE']).apply(lambda x: weighted_mean(x, 'pobre', 'factor07'))
C:\Users\User\AppData\Loc

## 4. Exploración distancia a comisarías

In [21]:
pathCom = "C:/Users/User/OneDrive - Universidad del Pacífico/1. Documentos/0. Bases de datos/11. Mapas Perú/comisarias_shapefile"

peruCom = gdp.read_file(pathCom + "/comisarias_fuente_cenepred.shp")

In [22]:
comisarias = peruCom.centroid

var1 = peruCom['nombre']
var2 = peruCom['dpto']
var3 = peruCom['dist']
var4 = peruCom['prov']
xCom = comisarias.x
yCom = comisarias.y
comisarias = pd.DataFrame({'nombreCom': var1,
                          'xCom' : xCom,
                          'yCom' : yCom,
                           'distCom' : var3,
                           'provCom' : var4,
                           'dptoCom' : var2})

del var1, xCom, yCom, var2, var3, var4

C:\Users\User\AppData\Local\Temp\ipykernel_20436\3087554850.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  comisarias = peruCom.centroid


In [23]:
comisarias.loc[comisarias['dptoCom'] == "AMAZONAS - CAJAMARCA",'dptoCom'] = "AMAZONAS"
comisarias.loc[comisarias['dptoCom'] == "AMAZONAS - SAN MARTIN",'dptoCom'] = "AMAZONAS"
comisarias.loc[comisarias['dptoCom'] == "JUNIN",'dptoCom']  = "JUNÍN"

comisarias = comisarias.sort_values(by='dptoCom')
comisarias['CCDD'], unique = pd.factorize(comisarias['dptoCom'])
comisarias['CCDD'] = comisarias['CCDD'] + 1

In [24]:
comisarias.loc[:,'CCDD'] = comisarias.loc[:,'CCDD'].astype(int).astype(str).str.zfill(2)

C:\Users\User\AppData\Local\Temp\ipykernel_20436\1274969275.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['01' '01' '01' ... '25' '25' '25']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  comisarias.loc[:,'CCDD'] = comisarias.loc[:,'CCDD'].astype(int).astype(str).str.zfill(2)


In [25]:
mod1.loc[:,'CCDD'] = (mod1.loc[:,'ubigeo'].astype(int)/10000).astype(int).astype(str).str.zfill(2)

In [26]:
mod1Com = pd.merge(mod1, comisarias, on = 'CCDD', how= 'inner', validate = 'm:m')

In [27]:
mod1Com = mod1Com.rename(columns={'latitud': 'latCong',
                                  'longitud': 'longCong',
                                  'yCom': 'latCom',
                                  'xCom': 'longCom'})

mod1Com = mod1Com.dropna(subset=['latCong', 'longCong', 'latCom', 'longCom'])

mod1Com.loc[:, 'distanciaCom'] = mod1Com.apply(
    lambda row: geodesic((row['latCong'], row['longCong']), (row['latCom'], row['longCom'])).kilometers, axis=1)

In [28]:
mod1Com2 = mod1Com.groupby(['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'pobreza', 'factor07']).min(
    'distanciaCom')
mod1Com2 = mod1Com2.reset_index()

In [29]:
# Definir los límites de los intervalos
bins = [0, 0.5, 1, 2.5, 500]

# Definir los nombres de las categorías
labels = ['Menos de 500m', 'Entre 0.5 y 1Km', 'Entre 1 y 2.5 Km', 'Más de 2.5Km']

# Crear la nueva columna categórica
mod1Com2['distCom'] = pd.cut(mod1Com2['distanciaCom'], bins=bins, labels=labels, right=False)


In [54]:
mod1Com2['pobre'] = mod1Com2['pobreza'] < 3

pobLimaCom = mod1Com2[mod1Com2['estrato'] == 1].groupby(['anio', 'distCom']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobEst2Com = mod1Com2[mod1Com2['estrato'] == 2].groupby(['anio', 'distCom']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobRestoCom = mod1Com2[mod1Com2['estrato'] > 2].groupby(['anio', 'distCom']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))

C:\Users\User\AppData\Local\Temp\ipykernel_20436\2400760424.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pobLimaCom = mod1Com2[mod1Com2['estrato'] == 1].groupby(['anio', 'distCom']).apply(
C:\Users\User\AppData\Local\Temp\ipykernel_20436\2400760424.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pobLimaCom = mod1Com2[mod1Com2['estrato'] == 1].groupby(['anio', 'distCom']).apply(
C:\Users\User\AppData\Local\Temp\ipykernel_20436\2400760424.py:5: FutureWarning: The default of observed=False is depreca

## 5. Exploración distancia a EE.SS.

In [32]:
sumaria = pd.read_stata('sumaria.dta', convert_categoricals=False,
                        columns= ['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'percepho', 'mieperho', 'gashog2d',
                                  'linpe', 'linea', 'gashog1d', 'pobrezav', 'pobreza', 'ipcr_0', 'gpgru0'])

mod1 = pd.read_stata('modulo100.dta', convert_categoricals=False,
                     columns= ['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'latitud', 'longitud'])

mod4 = pd.read_stata('modulo400.dta', convert_categoricals=False,
                     columns= ['anio', 'mes', 'conglome', 'vivienda','hogar', 'ubigeo', 'dominio', 'estrato', 'codperso', 'factor07', 'p407g1', 'p407g2', 'p407g3'])

C:\Users\User\AppData\Local\Temp\ipykernel_20436\757857045.py:1: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  sumaria = pd.read_stata('sumaria.dta', convert_categoricals=False,
C:\Users\User\AppData\Local\Temp\ipykernel_20436\757857045.py:5: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  mod1 = pd.read_stata('modulo100.dta', convert_categoricals=False,
C:\Users\User\AppData\Local\Temp\ipykernel_20436\757857045.py:8: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8,

In [33]:
cond1 = mod4['estrato'] < 6
cond2 = mod4['anio'] >= 2018
mod4 = mod4[cond1 & cond2]

In [35]:
mod4 = pd.merge(mod4, mod1, on=['anio', 'mes', 'conglome', 'vivienda', 'hogar'], how='inner', validate='m:1')
mod4 = pd.merge(mod4, sumaria, on=['anio', 'mes', 'conglome', 'vivienda', 'hogar'], how='inner', validate='m:1')

### Según ubicación de CCSS público más cercano

In [ ]:
os.chdir(
    r'C:\Users\User\OneDrive - MIGRACIÓN VIDENZA\1. Proyectos\1. Proyectos actuales\23. Artículos PDB\2. PDB - Pobreza Urbana\2. Data\1. Bases')

dfCCSS = pd.read_excel('listado_ccss.xls')
dfCCSS.loc[:, 'UBIGEO'] = dfCCSS.loc[:, 'UBIGEO'].astype(str).str.zfill(6)

In [62]:
cond = (dfCCSS['Institución'] == 'GOBIERNO REGIONAL') | (dfCCSS['Institución'] == 'MINSA') | (dfCCSS['Institución'] == 'MUNICIPALIDAD PROVINCIAL') | (dfCCSS['Institución'] == 'MUNICIPALIDAD DISTRITAL')
dfCCSSMINSAGR = dfCCSS[cond]

In [ ]:
dfCCSSMINSAGR = dfCCSSMINSAGR[dfCCSSMINSAGR['NORTE'].notna()]
dfCCSSMINSAGR = dfCCSSMINSAGR.rename(columns={'NORTE' : 'lat_ccss',
                                              'ESTE' : 'long_ccss',
                                              'UBIGEO' : 'ubigeo'})

In [74]:
mod4CCSS = pd.merge(mod4, dfCCSSMINSAGR, on=['ubigeo'], how= 'inner', validate='m:m')

mod4CCSS = mod4CCSS.rename(columns={'latitud': 'latCong',
                                    'longitud': 'longCong'})
mod4CCSS = mod4CCSS[mod4CCSS['latCong'].notna()]


In [76]:
mod4CCSS.loc[:, 'distanciaCCSS'] = mod4CCSS.apply(lambda row: geodesic((row['latCong'], row['longCong']), (row['lat_ccss'], row['long_ccss'])).kilometers, axis=1)

In [85]:
condI = (mod4CCSS['Categoria'] == 'I-1') | (mod4CCSS['Categoria'] == 'I-2') | (mod4CCSS['Categoria'] == 'I-3') | (mod4CCSS['Categoria'] == 'I-4')
condII = (mod4CCSS['Categoria'] == 'II-E') | (mod4CCSS['Categoria'] == 'II-1') | (mod4CCSS['Categoria'] == 'II-2')

mod4CCSS_I = mod4CCSS[condI].groupby(['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'codperso', 'pobreza', 'factor07']).min('distanciaCCSS')
mod4CCSS_II = mod4CCSS[condII].groupby(['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'codperso', 'pobreza', 'factor07']).min('distanciaCCSS')
mod4CCSS_I = mod4CCSS_I.reset_index()
mod4CCSS_II = mod4CCSS_II.reset_index()

mod4CCSS_I_II = mod4CCSS[condI | condII].groupby(['anio', 'mes', 'conglome', 'vivienda', 'hogar', 'codperso', 'pobreza', 'factor07']).min('distanciaCCSS')
mod4CCSS_I_II = mod4CCSS_I_II.reset_index()


In [86]:
# Definir los límites de los intervalos
bins = [0, 0.5, 1, 2.5, 500]

# Definir los nombres de las categorías
labels = ['Menos de 500m', 'Entre 0.5 y 1Km', 'Entre 1 y 2.5 Km', 'Más de 2.5Km']

# Crear la nueva columna categórica
mod4CCSS_I['distCCSS'] = pd.cut(mod4CCSS_I['distanciaCCSS'], bins=bins, labels=labels, right=False)
mod4CCSS_II['distCCSS'] = pd.cut(mod4CCSS_II['distanciaCCSS'], bins=bins, labels=labels, right=False)
mod4CCSS_I_II['distCCSS'] = pd.cut(mod4CCSS_I_II['distanciaCCSS'], bins=bins, labels=labels, right=False)


In [87]:
mod4CCSS_I['pobre'] = mod4CCSS_I['pobreza'] < 3

pobLimaCCSS_I = mod4CCSS_I[mod4CCSS_I['estrato'] == 1].groupby(['anio', 'distCCSS']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobEst2CCSS_I = mod4CCSS_I[mod4CCSS_I['estrato'] == 2].groupby(['anio', 'distCCSS']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobRestoCCSS_I = mod4CCSS_I[mod4CCSS_I['estrato'] > 2].groupby(['anio', 'distCCSS']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))


mod4CCSS_II['pobre'] = mod4CCSS_II['pobreza'] < 3

pobLimaCCSS_II = mod4CCSS_II[mod4CCSS_II['estrato'] == 1].groupby(['anio', 'distCCSS']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobEst2CCSS_II = mod4CCSS_II[mod4CCSS_II['estrato'] == 2].groupby(['anio', 'distCCSS']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobRestoCCSS_II = mod4CCSS_II[mod4CCSS_II['estrato'] > 2].groupby(['anio', 'distCCSS']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))

mod4CCSS_I_II['pobre'] = mod4CCSS_I_II['pobreza'] < 3

pobLimaCCSS2 = mod4CCSS_II[mod4CCSS_I_II['estrato'] == 1].groupby(['anio', 'distCCSS']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobEst2CCSS2 = mod4CCSS_II[mod4CCSS_I_II['estrato'] == 2].groupby(['anio', 'distCCSS']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobRestoCCSS2 = mod4CCSS_II[mod4CCSS_I_II['estrato'] > 2].groupby(['anio', 'distCCSS']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))

C:\Users\User\AppData\Local\Temp\ipykernel_20436\974185844.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pobLimaCCSS_I = mod4CCSS_I[mod4CCSS_I['estrato'] == 1].groupby(['anio', 'distCCSS']).apply(
C:\Users\User\AppData\Local\Temp\ipykernel_20436\974185844.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pobLimaCCSS_I = mod4CCSS_I[mod4CCSS_I['estrato'] == 1].groupby(['anio', 'distCCSS']).apply(
C:\Users\User\AppData\Local\Temp\ipykernel_20436\974185844.py:5: FutureWarning: The default of observed=Fal

### Según INEI

In [40]:
mod4 = mod4[mod4['p407g1'].notna()]

In [43]:
mod4.loc[:, 'tiempoCCSS'] = mod4.apply(lambda row: row['p407g1']*24*60 + row['p407g2']*60 + row['p407g3'], axis=1)

In [53]:
# Definir los límites de los intervalos
bins = [0, 45, 90, 180, 22000]

# Definir los nombres de las categorías
labels = ['Menos de 0.5h', 'Entre 0.5 y 1.5h', 'Entre 1.5 y 3h', 'Más de 3h']

# Crear la nueva columna categórica
mod4['distCcssENAHO'] = pd.cut(mod4['tiempoCCSS'], bins=bins, labels=labels, right=False)

In [56]:
mod4['pobre'] = mod4['pobreza'] < 3

pobLimaCCSS1 = mod4[mod4['estrato'] == 1].groupby(['anio', 'distCcssENAHO']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobEst2CCSS1 = mod4[mod4['estrato'] == 2].groupby(['anio', 'distCcssENAHO']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))
pobRestoCCSS1 = mod4[mod4['estrato'] > 2].groupby(['anio', 'distCcssENAHO']).apply(
    lambda x: weighted_mean(x, 'pobre', 'factor07'))

C:\Users\User\AppData\Local\Temp\ipykernel_20436\3954695156.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  pobLimaCCSS1 = mod4[mod4['estrato'] == 1].groupby(['anio', 'distCcssENAHO']).apply(
C:\Users\User\AppData\Local\Temp\ipykernel_20436\3954695156.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pobLimaCCSS1 = mod4[mod4['estrato'] == 1].groupby(['anio', 'distCcssENAHO']).apply(
C:\Users\User\AppData\Local\Temp\ipykernel_20436\3954695156.py:5: FutureWarning: The default of observed=False is depreca

In [58]:
os.chdir(r'C:\Users\User\OneDrive - MIGRACIÓN VIDENZA\1. Proyectos\1. Proyectos actuales\23. Artículos PDB\2. PDB - Pobreza Urbana\2. Data\1. Bases')

dfCCSS = pd.read_excel('listado_ccss.xls')

In [59]:
dfCCSS.loc[:,'UBIGEO'] = dfCCSS.loc[:,'UBIGEO'].astype(str).str.zfill(6)

C:\Users\User\AppData\Local\Temp\ipykernel_20436\3197352406.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['150134' '070106' '250101' ... '050408' '090117' '150110']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dfCCSS.loc[:,'UBIGEO'] = dfCCSS.loc[:,'UBIGEO'].astype(str).str.zfill(6)
